In [13]:
import pandas as pd
import torch
import numpy as np
from sentence_transformers import SentenceTransformer

In [20]:
papers = pd.read_csv('../data/processed_data/scopus_data_doi_cleaned.csv')
papers.head(3)

,title,abstract,year,coverdate,publication_name,citation_count,subject_areas,author_names,affiliations,countries,file_name,doi,orig_text,text
0,Artificial neural network prediction of transp...,Novel N-methyl-4-piperidinol (MPDL)-based solv...,2022,2022-08-01,Energy Reports,4.0,Energy (all),Nimmanterdwong P.+Janthboon P.+Tontiwachwuthik...,Chulalongkorn University+Clean Energy Technolo...,Thailand+Canada+China,202201532.json,10.1016/j.egyr.2022.02.117,Artificial neural network prediction of transp...,artificial neural network prediction transport...
1,Light-assisted synthesis of Au/TiO2 nanopartic...,A series of Au/TiO2 photocatalysts was synthes...,2022,2022-07-01,International Journal of Hydrogen Energy,20.0,"Renewable Energy, Sustainability and the Envir...",Kunthakudee N.+Puangpetch T.+Ramakul P.+Seriva...,Chulalongkorn University+Silpakorn University+...,Thailand,202201728.json,10.1016/j.ijhydene.2022.05.150,Light-assisted synthesis of Au/TiO2 nanopartic...,light assisted synthesis au tio2 nanoparticles...
2,Competition and epibiosis by the sponge Pseudo...,Competition for space on coral reefs is fierce...,2022,2022-01-01,Regional Studies in Marine Science,3.0,"Ecology, Evolution, Behavior and Systematics+A...",Allchurch A.+Mehrotra R.+Carmody H.+Monchanin ...,The University of Western Australia+Chulalongk...,Australia+Thailand+Canada+United States+France,202204089.json,10.1016/j.rsma.2021.102131,Competition and epibiosis by the sponge Pseudo...,competition epibiosis sponge pseudoceratina pu...


In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
MODEL_NAME = 'all-mpnet-base-v2'

model = SentenceTransformer(MODEL_NAME, device=device)
embeddings = model.encode(
    papers['orig_text'].tolist(), 
    show_progress_bar=True, 
    convert_to_numpy=True,
    batch_size=64
)

Batches:  54%|█████▎    | 145/271 [01:22<01:12,  1.75it/s]


KeyboardInterrupt: 

In [ ]:
embeddings.shape

(17340, 768)

In [ ]:
len(papers.text.to_list())

17340

In [ ]:
import faiss

np.save('../models/scopus_embeddings.npy', embeddings)

dimension = embeddings.shape[1]
ids = np.arange(embeddings.shape[0])

index = faiss.IndexFlatL2(dimension)

index_idx = faiss.IndexIDMap(index)
index_idx.add_with_ids(embeddings, ids)

faiss.write_index(index_idx, '../models/faiss_scopus_index.idx')